In [1]:
import pandas as pd

In [4]:
skills_trust = pd.read_csv('preprocessing/even_smaller_skills_trust.csv') # skills_relevance
skills_trust = skills_trust.drop(columns=['Unnamed: 0'])
skills_score = pd.read_csv('preprocessing/even_smaller_skills_score.csv') # skills_score
skills_score = skills_score.drop(columns=['Unnamed: 0'])
exp_trust = pd.read_csv('preprocessing/extracted_resume_sections_formatted_vague.csv') # experience_vague
reco_sentiments = pd.read_csv('preprocessing/added_lor_sentiments.csv') # lor_sentiment_score
reco_trust = pd.read_csv('preprocessing/added_lor_trust_score.csv') # lor_trust_score

In [27]:
combined = exp_trust.copy()
combined['skills_trust'] = skills_trust['skills_relevance']
combined['skills_score'] = skills_score['skills_skore']
combined['reco_trust'] = reco_trust['lor_trust_score']
combined['reco_score'] = reco_sentiments['lor_sentiment_score']
combined['lor_from'] = reco_sentiments['lor_from']

In [28]:
combined.drop(columns=['Resume ID'], inplace=True)

In [30]:
combined.to_csv('combined.csv')

In [1]:
import pandas as pd

combined = pd.read_csv('combined.csv')

In [2]:
combined.columns

Index(['Unnamed: 0', 'id', 'licenses', 'accomplishments',
       'personal information', 'salary', 'languages', 'training', 'summary',
       'publications', 'education', 'contact information', 'volunteer',
       'experience', 'interests', 'skills', 'experience_vague', 'skills_trust',
       'skills_score', 'reco_trust', 'reco_score', 'lor_from'],
      dtype='object')

In [5]:
combined["role"] = reco_sentiments["role"]

In [9]:
map_col = {"id": "name",
           "role": "role",
           "skills_trust": "skillTrust",
           "skills_score": "skillScore",
           "reco_trust": "recoTrust",
           "reco_score": "recoScore",
           "experience_vague": "experienceAccuracy"}

new_df = pd.DataFrame(columns=map_col.values())
for key in map_col.keys():
    new_df[map_col[key]] = combined[key]

new_df.head()

,name,role,skillTrust,skillScore,recoTrust,recoScore,experienceAccuracy
0,10,Operations Manager,4.0,4.0,4.0,4.0,4.0
1,103,Accountant,4.0,4.0,5.0,4.0,2.0
2,106,Finance Director,5.0,4.0,5.0,5.0,4.0
3,107,Accountant,4.0,3.0,5.0,4.0,2.0
4,108,Fitness Instructor,1.0,2.0,5.0,4.0,4.0


In [11]:
# convert the name field to string
new_df["name"] = new_df["name"].astype(str)
new_df.to_csv('combined_mongo_users.csv', index=False)

In [13]:
roles_skills = {
    "Accountant": ["MS Excel", "Accounting Software (like Sage)", "Financial Reporting"],
    "Human Resources Specialist": ["Mediation", "Recruitment Planning", "Complaints Management"],
    "Finance Director": ["Investor Relations", "Financial Planning", "Risk Management"],
    "Director/VP of Information Technology": ["Leadership", "Project Management", "Technical Skills", "Highly Experienced"],
    "Operations Manager": ["Supply Chain Management", "Process Optimization", "Highly Experienced"],
    "Fitness Instructor": ["Fitness Training Experience", "Yoga", "Self Defense"],
    "Finance Manager": ["Communication skills", "MS Office", "Sales and Marketing"],
    "Customer Service Representative": ["Communication", "Management Skills", "Problem Solving"],
    "HR Director": ["Organizational Development", "Change Management", "Leadership"],
    "Finance Analyst": ["MS Excel", "Accounting Software (like Sage)", "Financial Reporting"],
    "Information Technology Manager" : ["Leadership", "Project Management", "Technical Skills"]
}

# convert to df with columns role and expectations
role_expectations = pd.DataFrame(columns=["role", "expectations"])
for role in roles_skills.keys():
    new_row_df = pd.DataFrame({"role": [role], "expectations": [roles_skills[role]]})
    role_expectations = pd.concat([role_expectations, new_row_df])

role_expectations.to_csv('role_expectations_mongo.csv', index=False)